<a href="https://colab.research.google.com/github/johngcarlsson-blai/roulette/blob/main/Roulette_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import random
import ipywidgets as widgets
from IPython.display import display, clear_output

# American roulette numbers including 00
roulette_wheel = ['00'] + list(range(0, 37))

# Columns for 'middle' and 'third' profiles
columns_profiles = {
    'middle': [2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35],
    'third': [3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36]
}

# Double bets for 'middle' and 'third' profiles
double_bets_profiles = {
    'middle': [12, 25],
    'third': [11, 22]
}

# Single bets for 'middle' and 'third' profiles
single_bets_profiles = {
    'middle': [0, 3, 7, 10, 28, 30],
    'third': [1, 4, 13, 14, 16, 19]
}

# Define the bet levels
unit_levels = [0, 2, 4, 6, 8, 12, 16, 20, 24, 30]

def get_next_level(current_level, step):
    """Get the next unit level based on the step (up or down by 1 in the list)."""
    current_index = unit_levels.index(current_level)
    next_index = min(max(current_index + step, 0), len(unit_levels) - 1)  # Ensure within bounds
    return unit_levels[next_index]

def switch_profile(current_profile):
    """Switch between 'middle' and 'third' profiles."""
    return 'third' if current_profile == 'middle' else 'middle'

def print_bets(column_bet, double_bet, single_bet):
    print(f"\nColumn Bet: ${column_bet}, Double Bets: ${double_bet}, Single Bets: ${single_bet}")

def simulate_betting_system(rolls, profile='middle'):
    # Set initial profile and bets
    current_profile = profile
    column_numbers = columns_profiles[current_profile]
    double_bet_numbers = double_bets_profiles[current_profile]
    single_bet_numbers = single_bets_profiles[current_profile]

    # Initial bets
    base_unit = 12.50
    unit_size = 8  # Start at 8 units
    column_bet = unit_size * base_unit * 24
    double_bet = column_bet // 12  # Double bet scaled by 2/24
    single_bet = column_bet // 24  # Single bet scaled by 1/24

    first_bet = True  # Track if it's the first bet
    consecutive_losses = 0  # Track consecutive losses
    paused = False  # Track if betting is paused after 3 consecutive losses

    total_earnings = 0  # Track total earnings

    for i, result in enumerate(rolls):
        # Check if the profile should switch (only if the dynamic profile is selected)
        if profile == 'dynamic' and total_earnings > 21000:
            current_profile = switch_profile(current_profile)
            column_numbers = columns_profiles[current_profile]
            double_bet_numbers = double_bets_profiles[current_profile]
            single_bet_numbers = single_bets_profiles[current_profile]
            print(f"\nTotal earnings exceed 21,000. Switching profile to '{current_profile}'.")

        if total_earnings >= 14000:
          print(f"Up by $14,000 or more. Resetting unit size to 2 units.")
          unit_size = 2
          column_bet = unit_size * base_unit * 24
          double_bet = column_bet // 12
          single_bet = column_bet // 24

        print_bets(column_bet, double_bet, single_bet)
        print(f"Roll {i+1}: The ball landed on {result}")

        # Keep track of net result for this round
        net_result = 0

        # Check if the result is a win for column, double, or single bets
        column_win = int(result) in column_numbers if result not in ['00', '0'] else False
        double_win = int(result) in double_bet_numbers if result not in ['00', '0'] else False
        single_win = int(result) in single_bet_numbers if result not in ['00', '0'] else False

        total_bet = column_bet + 2 * double_bet + 6 * single_bet

        if paused:
            # Check if a single or double win occurs while betting is paused
            if column_win or double_win or single_win:
                print("A win occurred while betting was paused. Resuming betting.")
                print(f"Running total: ${total_earnings}")
                paused = False  # Resume betting
            else:
                print("Still paused due to consecutive losses.")
            continue  # Skip to the next roll if paused

        total_earnings -= total_bet  # Deduct the total bet

        # Track consecutive losses
        if not (column_win or double_win or single_win):
            consecutive_losses += 1
        else:
            consecutive_losses = 0  # Reset if there's a win

        # Rule: Pause betting after 3 consecutive losses
        if consecutive_losses >= 3:
            print("Pausing betting due to 3 consecutive losses")
            print(f"Running total: ${total_earnings}")
            paused = True
            continue  # Skip placing bets but continue checking rolls

        # Handle win/loss logic
        if column_win:
            print(f"Column bet wins! ({result})")
            net_result = (2 * column_bet) + column_bet  # 2:1 payout for column bet
            if first_bet:  # Double only on the first bet win
                unit_size = get_next_level(unit_size, 1)  # Move up one level
                column_bet = unit_size * base_unit * 24
                double_bet = column_bet // 12
                single_bet = column_bet // 24
                first_bet = False  # After first bet, no more doubling
            else:
                # After the first bet, just increase the bet size by 1 level
                unit_size = get_next_level(unit_size, 1)
                column_bet = unit_size * base_unit * 24
                double_bet = column_bet // 12
                single_bet = column_bet // 24
        elif double_win:
            print(f"Double bet wins! ({result})")
            net_result = (35 * double_bet) + double_bet  # 35:1 payout for double bet
            if first_bet:  # Double only on the first bet win
                unit_size = get_next_level(unit_size, 1)  # Move up one level
                column_bet = unit_size * base_unit * 24
                double_bet = column_bet // 12
                single_bet = column_bet // 24
                first_bet = False  # After first bet, no more doubling
            else:
                # After the first bet, just increase the bet size by 1 level
                unit_size = get_next_level(unit_size, 1)
                column_bet = unit_size * base_unit * 24
                double_bet = column_bet // 12
                single_bet = column_bet // 24
        elif single_win:
            print(f"Single bet wins! ({result})")
            net_result = (35 * single_bet) + single_bet  # 35:1 payout for single bet
            if unit_size == 0:
                unit_size = 2  # If at zero, go to 2 units after a single win
                column_bet = unit_size * base_unit * 24
                double_bet = column_bet // 12
                single_bet = column_bet // 24
            # Otherwise, keep the bet the same
            first_bet = False  # After first bet, no more doubling
        else:
            print(f"No win, all bets lost on roll {i+1}")
            # Go down by 1 level after a loss
            unit_size = get_next_level(unit_size, -1)
            column_bet = unit_size * base_unit * 24
            double_bet = column_bet // 12
            single_bet = column_bet // 24
            first_bet = False  # After the first bet, no more doubling

        # Ensure no betting occurs when unit size is 0
        if unit_size == 0:
            column_bet = 0
            double_bet = 0
            single_bet = 0

        # Ensure minimum bet isn't below a threshold (e.g., $100 total)
        if column_bet < 100 and unit_size != 0:  # Allow betting 0, but don't go below $100 otherwise
            column_bet = 100
            double_bet = column_bet // 12
            single_bet = column_bet // 24

        # Update total earnings
        total_earnings += net_result
        print(f"Running total: ${total_earnings}")

    # Print final total
    print(f"\nFinal total earnings: ${total_earnings}")


# Example usage:
# roll_sequence = [22,24,30,31,2,34,32,5,0,24,20,20,0,25,16,12,0,24,36]
# simulate_betting_system(roll_sequence, profile='middle')  # Dynamic profile will switch if total earnings exceed 21,000

# Create the input widgets
rolls_input = widgets.Text(
    value='',
    placeholder='Enter roll values separated by commas',
    description='Rolls:',
    disabled=False
)

run_button = widgets.Button(
    description='Run Simulation',
    disabled=False,
    button_style='success'
)

output_area = widgets.Output()

# Event handler for the button click
def on_run_button_clicked(b):
    # Clear previous output
    with output_area:
        clear_output()
        # Get the input rolls from the text box
        rolls_str = rolls_input.value
        try:
            # Convert the string input to a list of integers
            rolls = [int(r.strip()) if r.strip() not in ['00'] else '00' for r in rolls_str.split(',')]
            simulate_betting_system(rolls, profile='middle')  # You can change the profile here if needed
        except ValueError:
            print("Invalid input. Please enter valid roll values.")

# Bind the button to the event handler
run_button.on_click(on_run_button_clicked)

# Display the widgets
display(rolls_input, run_button, output_area)

Text(value='', description='Rolls:', placeholder='Enter roll values separated by commas')

Button(button_style='success', description='Run Simulation', style=ButtonStyle())

Output()